In [118]:
import os
import civis
import requests
import json
import pandas as pd
import urllib.parse
from pathlib import Path
from datetime import date
from datetime import timedelta

# GLOBAL VARIABLES #
YESTERDAY_DATE = (date.today() - timedelta(days = 1)).strftime("%m/%d/%Y")
AFF_SYNC_COLUMNS = ['affiliateid','affiliateein','affiliatepercapitapin','IsChartered','CharterDate','electionmonth','officertermstartmonth','iselectionyearodd','electiontermyears','updatedby','updatedat']
APC_SYNC_COLUMNS = ['AffiliatePerCapitaId','FiscalYearEndMonth','FiscalYearEndDay','PayPerCapitaToAFT','InvoicedByAFT','IncludeAFLCIOPerCapita','AFLCIOAmount','AffiliateBillingFrequencyId','HasOccupationalLiabilityInsurance','FiduciaryBondCoverage','AccidentInsuranceUnits','ConventionDelegationEligibility','IsAgencyFee','IsStateDues','DeliveryType','GroupNumber','UpdatedBy','UpdatedAt','DeletedAt']
DATE_TO_RUN = '10/28/2022' #YESTERDAY_DATE

TABLES = ['Affiliate','AffiliatePerCapita','Accounts','StateFederation','AffiliateType','AffiliateDesignation','AffiliateGeoReach']

# JSON PRINT HELPER #
def jprint(output):
    print(json.dumps(output, indent=4))

class KnackAFT:
    def __init__(self):
        # API #
        self.API_KEY = '1b8065b3-d5a5-4586-946a-d9f5d315963f'
        self.APP_ID = '6157aca138a38604ae371cd9'
        
        # HTTP REQUESTS #
        self.GET_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID}
        self.POST_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID,'content-type':'application/json'}
        self.API_URL = f'https://api.knack.aft.org/v1/'
        self.LOADER_URL = f'https://loader.knack.aft.org/v1/applications/{self.APP_ID}'

        # INTERNAL #
        self.APP_DICT = {}

        res = requests.get(url=self.LOADER_URL)
        objects = res.json()['application']['objects']

        for obj in objects:
            fields = {}
            name = obj['name']
            key = obj['key']
            
            if name in TABLES:
                for item in obj['fields']:
                    fields.update({item['name']:item['key']})
                self.APP_DICT.update({name:{'obj_id':key,'fields':fields}})
        
    # function to return key for any value
    def get_key(self, dictionary ,val):
        for key, value in dictionary.items():
            if val == value:
                return key

        return ''
        
    
    # GET and format json from requestURL
    def getJSON(self, url):
        r = requests.get(url = self.API_URL + url, headers = self.GET_HEADERS)
        return r.json()
    
    def getObjectJSON(self, object_name):
        return (self.getJSON('objects/' + self.APP_DICT[object_name]['obj_id']))['object']
        
    def find_matches(self, object_name, field_name, match_val):
        field_id = self.APP_DICT[object_name]['fields'][field_name]
        object_id = self.APP_DICT[object_name]['obj_id']
        
        match_filter = {'match':'and', 'rules':[{'field':field_id, 'operator':'is', 'value': match_val}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "objects/" + object_id + "/records?filters=" + filter_for_url
        res = self.getJSON(request_url)
        if res["total_records"] == 0:
            return ''
        else:
            return res["records"]
        
    def find_records_updated_at_date(self, knack_object, date):
        #Convert to IDs
        knack_object_id = self.APP_DICT[knack_object]['obj_id']
        field_to_match_id = self.APP_DICT[knack_object]['fields']['mdate']

        #Get Id
        match_filter = {'match':'and', 'rules':[{'field':field_to_match_id, 'operator':'is', 'value': date}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url

        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 0:
            return ''
        else:
            return res_json_dict["records"]
        
    def get_userid(self, user):
        #Convert to IDs
        knack_object_id = self.APP_DICT['Accounts']['obj_id']
        knack_field_id = self.APP_DICT['Accounts']['fields']['KnackUserID']

        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records/" +  user
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        return res_json_dict[knack_field_id]
    
    def get_connection(self,connection_name,field,connection):
        #Convert to IDs
        knack_object_id = self.APP_DICT[connection_name]['obj_id']
        knack_field_id = self.APP_DICT[connection_name]['fields'][field]

        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records/" +  connection
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        return res_json_dict[knack_field_id]



client = KnackAFT()
aff_fields = {}
apc_fields = {}

for name in AFF_SYNC_COLUMNS:
    aff_fields.update({name:client.APP_DICT['Affiliate']['fields'][name]})
        
for name in APC_SYNC_COLUMNS:
    apc_fields.update({name:client.APP_DICT['AffiliatePerCapita']['fields'][name]})


def affiliate_runner():
    output_df = []
    records = client.find_records_updated_at_date('Affiliate', DATE_TO_RUN)
    if records:
        for record in records:
            output = {}
            for k,v in aff_fields.items():
                output.update({k:str(record[v])})
                
            user_conn = record[client.APP_DICT['Affiliate']['fields']['muser']+'_raw'][0]['id']
            mdate = record[client.APP_DICT['Affiliate']['fields']['mdate']]
            muser = client.get_userid(user_conn)
            
            val = client.get_connection('StateFederation','AffiliateId',record[client.APP_DICT['Affiliate']['fields']['ParentAffiliateID']+'_raw'][0]['id'])
            output.update({'ParentAffiliateID':val})
        
            val = client.get_connection('AffiliateType','AffiliateTypeId',record[client.APP_DICT['Affiliate']['fields']['AffiliateTypeID']+'_raw'][0]['id'])
            output.update({'AffiliateTypeID':val})
            
            val = client.get_connection('AffiliateDesignation','AffiliateDesignationId',record[client.APP_DICT['Affiliate']['fields']['AffiliateDesignationID']+'_raw'][0]['id'])
            output.update({'AffiliateDesignationID':val})
            
            val = client.get_connection('AffiliateGeoReach','AffiliateGeoReachId',record[client.APP_DICT['Affiliate']['fields']['AffiliateGeoReachID']+'_raw'][0]['id'])
            output.update({'AffiliateGeoReachID':val})
            
            output.update({'updatedat':str(mdate)})
            output.update({'updatedby':str(muser)})
            output_df.append(output)

    return pd.DataFrame(output_df, dtype=str)

def affiliatepercapita_runner():
    output_df = []
    records = client.find_records_updated_at_date('AffiliatePerCapita', DATE_TO_RUN)
    if records:
        for record in records:
            output = {}
            for k,v in apc_fields.items():
                output.update({k:str(record[v])})
                
            user_conn = record[client.APP_DICT['AffiliatePerCapita']['fields']['muser']+'_raw'][0]['id']
            mdate = record[client.APP_DICT['AffiliatePerCapita']['fields']['mdate']]
            muser = client.get_userid(user_conn)
            
            output.update({'UpdatedAt':str(mdate)})
            output.update({'UpdatedBy':str(muser)})
            output_df.append(output)
            
    return pd.DataFrame(output_df, dtype=str)

#jprint(client.APP_DICT)
    
def main(entity):
    if entity == 'aff':
        table_suffix = 'SYS_ARTS_KnackAffiliate_XN'
        output_records = affiliate_runner()
    elif entity == 'apc':
        table_suffix = 'SYS_ARTS_KnackAffiliatePerCapita_XN'
        output_records = affiliatepercapita_runner()
        
    rows = output_records.shape[0]
    columns = output_records.shape[1]

    if rows == 0 or columns == 0:
        print('No Changes found for '+entity+'!')
        print('Skipping write to civis...')
    else:
        print('Found '+str(rows)+' rows!')
        
        table_name = os.environ['civis_table_path'] + table_suffix
        print('Writing to civis table: ' + table_name + '...')
        
        fut = civis.io.dataframe_to_civis(output_records, 'American Federation of Teachers',table_name, existing_table_rows='append')
        fut.result()
        
        print('Done writing!')

print('==== Checking Affiliate...')
main('aff')
print('==== Checking AffiliatePerCapita...')
main('apc')


==== Checking Affiliate...
Found 6 rows!
Done writing!
==== Checking AffiliatePerCapita...
Found 1 rows!
Done writing!


In [1]:
import civis
import os
import requests
import json
import pandas as pd
import urllib.parse
from pathlib import Path
from datetime import date
from datetime import timedelta

# GLOBAL VARIABLES #
AFF_SYNC_COLUMNS = ['affiliateid','affiliateein','affiliatepercapitapin','IsChartered','CharterDate','electionmonth','officertermstartmonth','iselectionyearodd','electiontermyears','updatedby','updatedat']
APC_SYNC_COLUMNS = ['AffiliatePerCapitaId','FiscalYearEndMonth','FiscalYearEndDay','PayPerCapitaToAFT','InvoicedByAFT','IncludeAFLCIOPerCapita','AFLCIOAmount','AffiliateBillingFrequencyId','HasOccupationalLiabilityInsurance','FiduciaryBondCoverage','AccidentInsuranceUnits','ConventionDelegationEligibility','IsAgencyFee','IsStateDues','DeliveryType','GroupNumber','UpdatedBy','UpdatedAt','DeletedAt']
TABLES = ['Affiliate','AffiliatePerCapita','Accounts','StateFederation','AffiliateType','FiduciaryBondCoverage','AffiliateDesignation','AffiliateInactiveReason','AffiliateGeoReach']

# JSON PRINT HELPER #
def jprint(output):
    print(json.dumps(output, indent=4))

class KnackAFT:
    def __init__(self):
        # API #
        self.API_KEY = '1b8065b3-d5a5-4586-946a-d9f5d315963f'
        self.APP_ID = '6157aca138a38604ae371cd9'
        
        # HTTP REQUESTS #
        self.GET_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID}
        self.POST_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID,'content-type':'application/json'}
        self.API_URL = f'https://api.knack.aft.org/v1/'
        self.LOADER_URL = f'https://loader.knack.aft.org/v1/applications/{self.APP_ID}'

        # INTERNAL #
        self.APP_DICT = {}

        res = requests.get(url=self.LOADER_URL)
        objects = res.json()['application']['objects']

        for obj in objects:
            fields = {}
            name = obj['name']
            key = obj['key']
            
            if name in TABLES:
                for item in obj['fields']:
                    fields.update({item['name'].lower():item['key']})
                self.APP_DICT.update({name.lower():{'obj_id':key,'fields':fields}})
        
    # function to return key for any value
    def get_key(self, dictionary ,val):
        for key, value in dictionary.items():
            if val == value:
                return key

        return ''
        
    
    # GET and format json from requestURL
    def getJSON(self, url):
        r = requests.get(url = self.API_URL + url, headers = self.GET_HEADERS)
        return r.json()
    
    def getObjectJSON(self, object_name):
        return (self.getJSON('objects/' + self.APP_DICT[object_name]['obj_id']))['object']
        
    def find_matches(self, object_name, field_name, match_val):
        field_id = self.APP_DICT[object_name]['fields'][field_name]
        object_id = self.APP_DICT[object_name]['obj_id']
        
        match_filter = {'match':'and', 'rules':[{'field':field_id, 'operator':'is', 'value': match_val}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "objects/" + object_id + "/records?filters=" + filter_for_url
        res = self.getJSON(request_url)
        if res["total_records"] == 0:
            return ''
        else:
            return res["records"]
        
    def find_records_updated_at_date(self, knack_object, date):
        #Convert to IDs
        knack_object_id = self.APP_DICT[knack_object]['obj_id']
        field_to_match_id = self.APP_DICT[knack_object]['fields']['mdate']

        #Get Id
        match_filter = {'match':'and', 'rules':[{'field':field_to_match_id, 'operator':'is', 'value': date}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url

        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 0:
            return ''
        else:
            return res_json_dict["records"]
        
    def get_userid(self, user):
        #Convert to IDs
        knack_object_id = self.APP_DICT['Accounts']['obj_id']
        knack_field_id = self.APP_DICT['Accounts']['fields']['KnackUserID']

        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records/" +  user
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        return res_json_dict[knack_field_id]
    
    def get_connection(self,obj_name,field_name,connection):
        
        #Convert to IDs
        knack_object_id = self.APP_DICT[obj_name]['obj_id']
        knack_field_id = self.APP_DICT[obj_name]['fields'][field_name]

        #Get Id
        match_filter = {'match':'and', 'rules':[{'field':knack_field_id, 'operator':'is', 'value': connection}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 0:
            return ''
        elif res_json_dict["total_records"] == 1:
            return res_json_dict["records"][0]["id"]
        else:
            return ''



def add_affiliate(record):
    aff_dict = client.APP_DICT['affiliate']['fields']
    connection_fields = ["parentaffiliateid","affiliatetypeid","affiliatedesignationid","affiliategeoreachid","affiliateinactivereasonid"]
    out = {}
    
    for k,v in record.items():
        if k in connection_fields:
            if k == 'affiliateinactivereasonid':
                conn = client.get_connection('affiliateinactivereason','affiliateinactivereasonid',v)
                out.update({aff_dict['affiliateinactivereason']:conn})

            elif k == 'parentaffiliateid':
                conn = client.get_connection('affiliate','affiliateid',v)
                out.update({aff_dict[k]:conn})
            else:
                conn = client.get_connection(k.replace('id',''),k,v)
                out.update({aff_dict[k]:conn})

        else:
            out.update({aff_dict[k]:v})

    out.update({aff_dict['aftdbupdate']:1})
    check_exists = client.get_connection('affiliate', 'affiliateid', record['affiliateid'])
    if check_exists:
        print(record['affiliateid'] + " exists! updating record...")
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliate"]['obj_id'] + "/records/" +  check_exists
        r = requests.put(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
    else:
        print("Creating new affiliate with id: " + record['affiliateid'])
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliate"]['obj_id'] + "/records"
        r = requests.post(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        
        
def add_affiliatepercapita(record):
    record.pop('includestatepercapita')
    apc_dict = client.APP_DICT['affiliatepercapita']['fields']
    connection_fields = ['affiliateid', 'currentfiduciarybondcoverageid']
    out = {}
    
    for k,v in record.items():
        if k in connection_fields:
            if k == 'affiliateid':
                conn = client.get_connection('affiliate','affiliateid',v)
                out.update({apc_dict['affiliate']:conn})
            else:
                conn = client.get_connection('fiduciarybondcoverage','fiduciarybondcoverageid',v)
                out.update({apc_dict['fiduciarybondcoverage']:conn})

        else:
            out.update({apc_dict[k]:v})

    out.update({apc_dict['aftdbupdate']:1})
    check_exists = client.get_connection('affiliatepercapita', 'affiliatepercapitaid', record['affiliatepercapitaid'])
    if check_exists:
        print(record['affiliatepercapitaid'] + " exists! updating record...")
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliatepercapita"]['obj_id'] + "/records/" +  check_exists
        r = requests.put(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
    else:
        print("Creating new affiliate with id: " + record['affiliatepercapitaid'])
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliatepercapita"]['obj_id'] + "/records"
        r = requests.post(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        

def run_affiliate():
    table_name = os.environ['civis_table_path'] + 'affiliate'

    try:
        f = civis.io.read_civis(table=table_name,database="American Federation of Teachers",use_pandas=True)
        except civis.base.EmptyResultError as err:
            print('Empty upload table, aborting!...')
        else:
            f.fillna('', inplace=True)
            df = pd.DataFrame()
            df = f.astype(str)

            df['updatedat'] = pd.to_datetime(df.updatedat)
            df['updatedat'] = df['updatedat'].dt.strftime('%m/%d/%Y')
            
            df['createdat'] = pd.to_datetime(df.updatedat)
            df['createdat'] = df['createdat'].dt.strftime('%m/%d/%Y')
            
            df['deletedat'] = pd.to_datetime(df.updatedat)
            df['deletedat'] = df['deletedat'].dt.strftime('%m/%d/%Y')

            for payload_dict in df.to_dict('records'):
                add_affiliate(payload_dict)
                
def run_affiliatepercapita():
    table_name = os.environ['civis_table_path'] + 'affiliatepercapita'

    try:
        f = civis.io.read_civis(table=table_name,database="American Federation of Teachers",use_pandas=True)
        except civis.base.EmptyResultError as err:
            print('Empty upload table, aborting!...')
        else:
            f.fillna('', inplace=True)
            df = pd.DataFrame()
            df = f.astype(str)
            
            df['updatedat'] = pd.to_datetime(df.updatedat)
            df['updatedat'] = df['updatedat'].dt.strftime('%m/%d/%Y')
            
            df['createdat'] = pd.to_datetime(df.updatedat)
            df['createdat'] = df['createdat'].dt.strftime('%m/%d/%Y')
            
            df['deletedat'] = pd.to_datetime(df.updatedat)
            df['deletedat'] = df['deletedat'].dt.strftime('%m/%d/%Y')

            for payload_dict in df.to_dict('records'):
                add_affiliatepercapita(payload_dict)


client = KnackAFT()

print("Uploading affiliate...")
run_affiliate()
print("--------------------------------")

print("Uploading affiliatepercapita...")
run_affiliatepercapita()
print("--------------------------------")


3968 exists! updating record...
<Response [200]>
{
    "field_637": "3968",
    "field_635": "63658c9bacc287047b86bbdc",
    "field_638": "",
    "field_639": "",
    "field_640": "0",
    "field_641": "0",
    "field_642": "0",
    "field_643": "0.00",
    "field_644": "",
    "field_645": "0",
    "field_646": "0",
    "field_647": "0",
    "field_648": "0",
    "field_649": "0",
    "field_650": "0",
    "field_651": "0",
    "field_652": "0",
    "field_636": "6345e8f0536567265dcbbdd3",
    "field_653": "1",
    "field_654": "LARTST",
    "field_655": "0",
    "field_656": "0",
    "field_657": "0",
    "field_658": "DONOTSENDINVOICE",
    "field_659": "",
    "field_660": "",
    "field_661": "0",
    "field_662": "11/03/2022",
    "field_663": "0",
    "field_664": "11/03/2022",
    "field_665": "",
    "field_674": 1
}


In [4]:
df = pd.DataFrame({'DOB': {0: '2021-3-30', 1: '26/1/2016'}})
df['DOB'] = pd.to_datetime(df.DOB)
df['DOB1'] = df['DOB'].dt.strftime('%m/%d/%Y')
print(df.DOB1)

0    03/30/2021
1    01/26/2016
Name: DOB1, dtype: object


In [ ]:
apc_record = {
    'affiliatepercapitaid': '3968',
    'affiliateid': '10689',
    'fiscalyearendmonth': '',
    'fiscalyearendday': '',
    'paypercapitatoaft': '0',
    'invoicedbyaft': '0',
    'includestatepercapita': '0',
    'includeaflciopercapita': '0',
    'aflcioamount': '0.00',
    'affiliatebillingfrequencyid': '',
    'hasoccupationalliabilityinsurance': '0',
    'accidentinsuranceunits': '0',
    'hascope': '0',
    'iscopevoluntary': '0',
    'haspipe': '0',
    'isdirectbargaining': '0',
    'conventiondelegationeligibility': '0',
    'hasstatewebaccess': '0',
    'currentfiduciarybondcoverageid': '5',
    'requestedfiduciarybondcoverageid': '1',
    'customernumber': 'LARTST',
    'isagencyfee': '0',
    'isstatedues': '0',
    'isstateaflcio': '0',
    'deliverytype': 'DONOTSENDINVOICE',
    'insuranceagencyfee': '',
    'groupnumber': '',
    'createdby': '0',
    'createdat': '11/03/2022',
    'updatedby': '0',
    'updatedat': '11/03/2022',
    'deletedat': ''}


aff_record = {
    'affiliateid': '10689',
    'affiliateguid': 'DC646AC0-182E-466C-8DF2-B52C346ED566',
    'affiliatename': 'ARTS Test',
    'affiliatenumber': 'ARTST',
    'affiliateabbreviatedname': 'Arts Test Local',
    'affiliateacronym': 'ARTS',
    'affiliateein': '',
    'affiliatepercapitapin': '123456',
    'billhighwaygroupid': '',
    'ischartered': '1',
    'charterdate': '',
    'parentaffiliateid': '',
    'affiliatetypeid': '4',
    'affiliatedesignationid': '1',
    'affiliategeoreachid': '2',
    'isaffiliateactive': '1',
    'affiliateinactivereasonid': '',
    'affiliateinactivedate': '',
    'locationstateabr': '',
    'regionid': '',
    'retireeentitytypeid': '',
    'retireedestinationid': '',
    'electionmonth': '',
    'officertermstartmonth': '',
    'iselectionyearodd': '0',
    'electiontermyears': '',
    'noncoaupdate': '0',
    'nonationalupdate': '0',
    'nostateupdate': '0',
    'nolanwanupdate': '0',
    'noexternalupdate': '0',
    'affiliatewebsite': '',
    'isactionnetwork': '0',
    'usesaftmemberid': '0',
    'createdby': '0',
    'createdat': '2022-11-03',
    'updatedby': '0',
    'updatedat': '11/03/2022',
    'deletedat': ''}